# Fox: Eyetracking - Television

This notebook:

1. Preprocessing television conditions by

    a. aligning pupil labs data with the open bci event data

    b. extract pupil labs data for each ad, saving data associated with each ad

2. AOI Analyses starting with

    a. finding the television screen in the world video recordings from each advertisement

    b. creating an AOI based on the television screen defined from step 1

    c. aggregating pupil labs ET data for each frame of the world video recordings
    
    d. processes whether the fixation is inside or outside of the AOI
    

In [1]:
# libraries
import cv2
import numpy as np
import pandas as pd
import os
import csv
import av
import subprocess
import matplotlib.pyplot as plt
import glob
from datetime import datetime
from PIL import Image, ImageDraw
from scipy.interpolate import interp1d

In [3]:
# paths and files for subj looping
t7sheild = os.path.join('/Volumes/T7 Shield/fox')
subj_folders = glob.glob(os.path.join('/Volumes/T7 Shield/fox/et','sub-[0-9][0-9][0-9]_tv-*'))
subj_folders.sort()
print(len(subj_folders))
subj_folders

109


['/Volumes/T7 Shield/fox/et/sub-019_tv-lego-42b0cb7d',
 '/Volumes/T7 Shield/fox/et/sub-021_tv-pickers-97606087',
 '/Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780',
 '/Volumes/T7 Shield/fox/et/sub-023_tv-continental-594a5c2b',
 '/Volumes/T7 Shield/fox/et/sub-024_tv-fox-96dfc1c4',
 '/Volumes/T7 Shield/fox/et/sub-025_tv-nfl-a6e3e656',
 '/Volumes/T7 Shield/fox/et/sub-027_tv-lego-2e4ef2bf',
 '/Volumes/T7 Shield/fox/et/sub-028_tv-bigmood-76d32333',
 '/Volumes/T7 Shield/fox/et/sub-029_tv-pickers-bf2ffb67',
 '/Volumes/T7 Shield/fox/et/sub-030_tv-cnn-b8d33fbe',
 '/Volumes/T7 Shield/fox/et/sub-031_tv-continental-8c85bd3f',
 '/Volumes/T7 Shield/fox/et/sub-032_tv-fox-b5beca97',
 '/Volumes/T7 Shield/fox/et/sub-033_tv-nfl-3d751d85',
 '/Volumes/T7 Shield/fox/et/sub-034_tv-control-e9c00262',
 '/Volumes/T7 Shield/fox/et/sub-035_tv-lego-27d35985',
 '/Volumes/T7 Shield/fox/et/sub-036_tv-bigmood-2a7adbc3',
 '/Volumes/T7 Shield/fox/et/sub-037_tv-pickers-f8e0659f',
 '/Volumes/T7 Shield/fox/et/sub-038_tv-

In [4]:
# paths and files for testing
# locally
# subj_folder = os.path.join('data', 'sub-053_tv-lego-63478be7')
# video = os.path.join(subj_folder, 'a5ba9c89_0.0-813.692.mp4')
# events = pd.read_csv(os.path.join(subj_folder,'events.csv'))
# gaze = pd.read_csv(os.path.join(subj_folder,'gaze.csv'))
# world_timestamps = pd.read_csv(os.path.join(subj_folder,'world_timestamps.csv'))
# task_events = pd.read_csv('/Users/ebeard/Dropbox (Penn)/i3/foxmedia/data/tv_event/sub-053_lego/sub-053_task-events.csv')

# on t7shield
subj_folder = subj_folders[0]
big_video = os.path.join(subj_folder, '09e25ad2_0.0-784.762.mp4')
ad_folder = os.path.join(subj_folder, 'ads', '2')
ad_video = os.path.join(ad_folder, '2.mp4')
gaze_data_raw = os.path.join(ad_folder, 'gaze.csv')
gaze_data_avg = os.path.join(ad_folder, 'averaged_gaze_data.csv')
rectangle_data_raw = os.path.join(ad_folder, 'rectangle_data.csv')
filtered_rectangle_data = os.path.join(ad_folder, 'filtered_rectangle_data.csv')

# Preprocessing

In [5]:
def log_error(error_message, subj=None):
    today_date = datetime.now().strftime("%Y-%m-%d")
    log_filename = f"error_log-tv-{today_date}.txt"
    with open(log_filename, "a") as log_file:
        if subj:
            log_file.write(f"Subject: {subj}\n")
        log_file.write(f"Error: {error_message}\n\n")

## Subset Ad Block Videos
This is less relevant now, since we're going to block specifically into the ad videos

In [ ]:
# trying from https://gist.github.com/rennis250/6aa2bde829b58361fbd40ded8af9720c

# look for event just before onset
event_list = ['ad-1.begin', 'ad-1.end', 'ad-2.begin', 'ad-2.end']

for ad in event_list:

    event_name = ad
    event_id = event_name.split('.')[0].split('-')[-1]
    start_of_event = events[events['name'] == event_name]

    event_ts = start_of_event['timestamp [ns]'].values[0]
    frame_num = np.searchsorted(world_timestamps['timestamp [ns]'], event_ts)
    frame_num = int(frame_num)

    # find frame using pyav
    container = av.open(video)
    video_stream = container.streams.video[0]

    timestamp = frame_num * video_stream.time_base
    container.seek(int(timestamp * av.time_base), stream=video_stream)

    save_dir = os.path.join(subj_folder, f'adBlock_{event_id}')
    os.makedirs(save_dir, exist_ok=True)

    # save frame
    fc = 0
    for frame in container.decode(video=0):
        if fc == frame_num:
            frame = frame.to_ndarray(format="bgr24")
            save_path = os.path.join(save_dir, f'event_{frame_num}-{event_name}.jpg')
            cv2.imwrite(save_path, frame)
            break
            
        fc += 1

In [ ]:
# List of ad events
event_pairs = [
    ('ad-1.begin', 'ad-1.end'),
    ('ad-2.begin', 'ad-2.end')
]

# Check the duration of the input video using FFmpeg
def get_video_duration(video_path):
    ffmpeg_command = [
        'ffprobe',
        '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        video_path
    ]
    try:
        duration = subprocess.check_output(ffmpeg_command, stderr=subprocess.STDOUT).decode().strip()
        return float(duration)
    except subprocess.CalledProcessError as e:
        print("Error getting video duration:", e.output.decode())
        return None

# Get the video duration
video_duration = get_video_duration(video)
if video_duration is None:
    raise RuntimeError("Failed to retrieve video duration.")

# Get the base timestamp to normalize the timeline
base_timestamp = world_timestamps['timestamp [ns]'].iloc[0]
print(f"Base timestamp: {base_timestamp}")

# Process each ad event pair
for ad_begin, ad_end in event_pairs:

    ad_id = ad_begin.split('.')[0].split('-')[-1]
    save_dir = os.path.join(subj_folder, f'adBlock_{ad_id}')

    # Get the start and end timestamps for the ad in nanoseconds
    start_ts = events[events['name'] == ad_begin]['timestamp [ns]'].values[0]
    end_ts = events[events['name'] == ad_end]['timestamp [ns]'].values[0]

    # Sort the gaze data by filtering the start and end timestamps
    gaze_data = gaze[(gaze['timestamp [ns]'] >= start_ts) & (gaze['timestamp [ns]'] <= end_ts)]
    gaze_data.to_csv(os.path.join(save_dir, f'adBlock_{ad_id}_gaze.csv'), index=False)

    # Normalize timestamps to video timeline
    normalized_start_ts = start_ts - base_timestamp
    normalized_end_ts = end_ts - base_timestamp

    # Find the corresponding frame numbers
    start_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_start_ts + base_timestamp)
    end_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_end_ts + base_timestamp)

    # Convert normalized timestamps to time in seconds
    start_time = normalized_start_ts / 1e9
    end_time = normalized_end_ts / 1e9
    duration = end_time - start_time

    # Check if the duration matches the duration of the gaze_data
    gaze_duration = (gaze_data['timestamp [ns]'].iloc[-1] - gaze_data['timestamp [ns]'].iloc[0]) / 1e9
    if abs(duration - gaze_duration) > 0.01:
        print(f"Warning: Duration mismatch between video segment and gaze data. Video duration: {duration}, Gaze data duration: {gaze_duration}")

    # Debug: Print calculated times
    print(f"Event: {ad_begin} to {ad_end}")
    print(f"Start Time: {start_time}, End Time: {end_time}, Duration: {duration}")

    # Check if the start time and duration are valid
    if start_time < 0 or duration <= 0 or start_time + duration > video_duration:
        print(f"Invalid time range for {ad_begin} to {ad_end}. Skipping.")
        continue

    # Output file path
    output_path = os.path.join(save_dir, f'adBlock_{ad_id}.mp4')

    # FFmpeg command to extract the video segment
    ffmpeg_command = [
        'ffmpeg',
        '-i', video,
        '-ss', f'{start_time:.3f}',  # Start time with millisecond precision
        '-t', f'{duration:.3f}',     # Duration with millisecond precision
        '-c:v', 'libx264',           # Video codec
        '-c:a', 'aac',               # Audio codec
        '-strict', 'experimental',
        '-y',                        # Overwrite output file
        output_path
    ]

    # Execute the FFmpeg command
    try:
        subprocess.run(ffmpeg_command, check=True, stderr=subprocess.PIPE)
        print(f"Segment saved: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting segment {ad_begin} to {ad_end}: {e.stderr.decode()}")

print("Video processing completed.")


## Subset Individual Ads

### Align Pupil Labs with OpenBCI Event Markers

In [24]:
def align_events(events_df, world_timestamps_df, task_events_df):
    # Check if the input files are dataframes
    if not isinstance(events_df, pd.DataFrame) or not isinstance(world_timestamps_df, pd.DataFrame) or not isinstance(task_events_df, pd.DataFrame):
        raise ValueError("Input files must be dataframes.")

    # Filter out event_id 1
    task_events_df = task_events_df[task_events_df['event_id'] != 1].reset_index(drop=True)

    # Identify the 'ad-1.begin' timestamp in nanoseconds from events.csv
    ad_1_begin_timestamp = events_df.loc[events_df['name'] == 'ad-1.begin', 'timestamp [ns]'].values[0]
    frame_num = np.searchsorted(world_timestamps_df['timestamp [ns]'], ad_1_begin_timestamp)#, 'right')
    frame_num = int(frame_num)
    ad_1_begin_timestamp_world = world_timestamps_df['timestamp [ns]'].iloc[frame_num]

    # Calculate the adjusted onset times for event_ids using ad-1.begin as a reference
    # Note: Onset should be aligned so that event_id 2 matches ad-1.begin
    event_id_2_onset = task_events_df.loc[task_events_df['event_id'] == 2, 'onset'].values[0]
    event_id_2_onset_ns = (event_id_2_onset * (1_000/250)) * 1_000_000
    event_id_2_onset_ns = event_id_2_onset_ns.astype(np.int64)
    offset_ns = ad_1_begin_timestamp_world - event_id_2_onset_ns
    
    # Debugging: print offset and initial timestamp details
    # print(f"Offset (ns): {offset_ns}")
    # print(f"ad-1.begin timestamp: {ad_1_begin_timestamp}")
    # print(f"ad-1.begin timestamp world: {ad_1_begin_timestamp_world}")
    # print(f"Initial onset for event_id 2: {event_id_2_onset}")
    # print(f"Initial onset for event_id 2 (ns): {event_id_2_onset_ns}")

    # Adjust onset times for the remaining event_ids
    adjusted_onsets_ns = ((task_events_df['onset'] * (1_000/250)) * 1_000_000) + offset_ns
    adjusted_onsets_ns = adjusted_onsets_ns.astype(np.int64)

    # Debugging: print adjusted onsets
    # print("Adjusted Onsets (ns):")
    # print(adjusted_onsets_ns)

    # Find the nearest timestamps in world_timestamps.csv for each adjusted onset
    nearest_timestamps_ns = []

    for onset_ns in adjusted_onsets_ns:

        # Calculate differences to all world timestamps
        diffs = np.abs(world_timestamps_df['timestamp [ns]'] - onset_ns)

        # Find the index of the smallest difference
        nearest_index = diffs.argmin()
        
        # Get the nearest timestamp
        nearest_timestamp = world_timestamps_df['timestamp [ns]'].iloc[nearest_index]
        nearest_timestamps_ns.append(nearest_timestamp)

        # Debugging: print comparison details
        # print(f"Onset (ns): {onset_ns}, Nearest Timestamp (ns): {nearest_timestamp}, Difference (ns): {diffs[nearest_index]}")


    # Calculate durations in nanoseconds
    durations_ns = (task_events_df['duration'] * (1_000/250)) * 1_000_000

    # Create a DataFrame to display event IDs, adjusted onsets, nearest timestamps, and durations
    aligned_events_df = pd.DataFrame({
        'event_id': task_events_df['event_id'],
        'adjusted_onset_ns': adjusted_onsets_ns,
        'nearest_timestamp_ns': nearest_timestamps_ns,
        'duration_ns': durations_ns
    })

    return aligned_events_df

#### Testing

In [ ]:
# Example usage
aligned_events_df_2 = align_events(events, world_timestamps, task_events)
print(aligned_events_df_2)

In [ ]:
# Check data types
print(f"Data Type of Nearest Timestamps: {aligned_events_df_2['nearest_timestamp_ns'].dtype}")
print(f"Data Type of World Timestamps: {world_timestamps['timestamp [ns]'].dtype}")

# Ensure both are in np.int64
aligned_events_df_2['nearest_timestamp_ns'] = aligned_events_df_2['nearest_timestamp_ns'].astype(np.int64)
world_timestamps['timestamp [ns]'] = world_timestamps['timestamp [ns]'].astype(np.int64)


In [ ]:
# Check for matching timestamps with a tolerance
tolerance = 10  # nanoseconds

for idx, row in aligned_events_df_2.iterrows():
    if not any(np.isclose(row['nearest_timestamp_ns'], world_timestamps['timestamp [ns]'].values, atol=tolerance)):
        print(f"No match found for {row['nearest_timestamp_ns']} within tolerance.")

# To see all non-matches
non_matches = aligned_events_df_2[~aligned_events_df_2['nearest_timestamp_ns'].isin(world_timestamps['timestamp [ns]'])]
print("Non-matching timestamps:")
print(non_matches)


#### QC Checks

In [6]:
def plot_adjusted_onsets_vs_world_timestamps(aligned_events_df, world_timestamps_df, save_dir):
    # Extract values for plotting
    adjusted_onsets = aligned_events_df['adjusted_onset_ns']
    world_timestamps = world_timestamps_df['timestamp [ns]']

    # Calculate min and max of adjusted_onsets
    min_onset = adjusted_onsets.min()
    max_onset = adjusted_onsets.max()

    # Define the range within 5 values of min and max
    lower_bound = min_onset 
    upper_bound = max_onset 

    # Filter world_timestamps based on the defined range
    filtered_world_timestamps = world_timestamps[(world_timestamps >= lower_bound) & (world_timestamps <= upper_bound)]

    print(len(adjusted_onsets))
    print(len(world_timestamps))
    print(len(filtered_world_timestamps))

    # Plot adjusted onsets
    plt.figure(figsize=(14, 6))
    plt.scatter(adjusted_onsets, adjusted_onsets, color='red', label='Adjusted Onsets', marker='x')

    # Plot world timestamps
    plt.plot(filtered_world_timestamps, filtered_world_timestamps, label='World Timestamps', alpha=0.7)

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('')
    plt.title('Adjusted Onsets vs. World Timestamps')
    plt.legend()
    
    # Save the plot
    plot_path = os.path.join(save_dir, 'adjusted_onsets_vs_world_timestamps.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path

# Plot the data
#plot_adjusted_onsets_vs_world_timestamps(aligned_events_df_2, world_timestamps)

# we'll want to update these to save the plots into the correct folder


In [7]:
def plot_nearest_timestamps(aligned_events_df, save_dir):
    # Extract values for plotting
    adjusted_onsets = aligned_events_df['adjusted_onset_ns']
    nearest_timestamps = aligned_events_df['nearest_timestamp_ns']
    event_ids = aligned_events_df['event_id']

    # Plot nearest timestamps
    plt.figure(figsize=(14, 6))
    plt.scatter(adjusted_onsets, event_ids, color='blue', label='Adjusted Onsets', marker='o', alpha=0.8)
    plt.scatter(nearest_timestamps, event_ids, color='green', label='Nearest Timestamps', marker='x')

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('Event ID')
    plt.title('Adjusted Onsets and Nearest Timestamps per Event')
    plt.legend()
    
    # Save the plot
    plot_path = os.path.join(save_dir, 'adjusted_onsets_nearest_timestamps.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path

# Plot the nearest timestamps
# plot_nearest_timestamps(aligned_events_df_2)

In [8]:
def plot_event_durations(aligned_events_df, save_dir):
    # Extract values for plotting
    event_ids = aligned_events_df['event_id']
    adjusted_onsets = aligned_events_df['adjusted_onset_ns']
    durations = aligned_events_df['duration_ns']

    # Calculate end times based on durations
    adjusted_ends = adjusted_onsets + durations

    # Plot event durations
    plt.figure(figsize=(14, 6))
    for i, event_id in enumerate(event_ids):
        plt.plot([adjusted_onsets[i], adjusted_ends[i]], [event_id, event_id], marker='|', label=f'Event {event_id}' if i == 0 else "")

    # Labels and legend
    plt.xlabel('Timestamps (ns)')
    plt.ylabel('Event ID')
    plt.title('Event Durations and Onsets')
    plt.legend()
    
    # Save the plot
    plot_path = os.path.join(save_dir, 'event_durations_and_onsets.png')
    plt.savefig(plot_path)
    plt.close()

    return plot_path
# Plot the event durations
# plot_event_durations(aligned_events_df_2)


In [ ]:
# New code to handle aligned_events_df_2
# Save frames from aligned events (for QC/testing)
# find frame using pyav
container = av.open(video)
video_stream = container.streams.video[0]

save_dir = os.path.join(subj_folder, 'ads')
os.makedirs(save_dir, exist_ok=True)

for index, row in aligned_events_df_2.iterrows():

    event_id = row['event_id']
    nearest_timestamp_ns = row['nearest_timestamp_ns']
    duration = row['duration_ns']

    # Calculate the second timestamp
    second_timestamp_ns = nearest_timestamp_ns + duration

    # Find the frame numbers for both timestamps
    frame_num_1 = np.searchsorted(world_timestamps['timestamp [ns]'], nearest_timestamp_ns)
    frame_num_2 = np.searchsorted(world_timestamps['timestamp [ns]'], second_timestamp_ns)

    frame_num_1 = int(frame_num_1)
    frame_num_2 = int(frame_num_2)

    # Save both frames
    for frame_num in [frame_num_1, frame_num_2]:
        timestamp = frame_num * video_stream.time_base
        container.seek(int(timestamp * av.time_base), stream=video_stream)

        fc = 0
        for frame in container.decode(video=0):
            if fc == frame_num:
                frame = frame.to_ndarray(format="bgr24")
                save_path = os.path.join(save_dir, f'event-{int(event_id)}_{frame_num}.jpg')
                cv2.imwrite(save_path, frame)
                break
                
            fc += 1

### Extract Pupil Labs Videos and Eye Gaze Data

In [45]:
def process_video_segments(video, world_timestamps, aligned_events_df, gaze, save_dir):
    # Check the duration of the input video using FFmpeg
    def get_video_duration(video_path):
        ffmpeg_command = [
            'ffprobe',
            '-v', 'error',
            '-show_entries', 'format=duration',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            video_path
        ]
        try:
            duration = subprocess.check_output(ffmpeg_command, stderr=subprocess.STDOUT).decode().strip()
            return float(duration)
        except subprocess.CalledProcessError as e:
            error_message = f"Error getting video duration: {e.output.decode()}"
            print(error_message)
            log_error(error_message)
            return None

    # Get the video duration
    video_duration = get_video_duration(video)
    if video_duration is None:
        raise RuntimeError("Failed to retrieve video duration.")

    # Get the base timestamp to normalize the timeline
    base_timestamp = world_timestamps['timestamp [ns]'].iloc[0]
    # print(f"Base timestamp: {base_timestamp}")

    # Process each ad 
    for index, row in aligned_events_df.iterrows():
        event_id = row['event_id']

        if event_id == 6:
            continue

        ad_save_dir = os.path.join(save_dir, f'{int(event_id)}')
        os.makedirs(ad_save_dir, exist_ok=True)

        nearest_timestamp_ns = row['nearest_timestamp_ns']
        duration = row['duration_ns']

        # Calculate the second timestamp
        second_timestamp_ns = nearest_timestamp_ns + duration

        # Find the frame numbers for both timestamps
        frame_num_1 = np.searchsorted(world_timestamps['timestamp [ns]'], nearest_timestamp_ns)
        frame_num_2 = np.searchsorted(world_timestamps['timestamp [ns]'], second_timestamp_ns)

        frame_num_1 = int(frame_num_1)
        frame_num_2 = int(frame_num_2)

        # Get the start and end timestamps for the ad in nanoseconds
        start_ts = world_timestamps['timestamp [ns]'].iloc[frame_num_1]
        end_ts = world_timestamps['timestamp [ns]'].iloc[frame_num_2]

        # Sort the gaze data by filtering the start and end timestamps
        gaze_data = gaze[(gaze['timestamp [ns]'] >= start_ts) & (gaze['timestamp [ns]'] <= end_ts)]
        gaze_data.to_csv(os.path.join(ad_save_dir, f'gaze.csv'), index=False)

        # Normalize timestamps to video timeline
        normalized_start_ts = start_ts - base_timestamp
        normalized_end_ts = end_ts - base_timestamp

        # Find the corresponding frame numbers
        start_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_start_ts + base_timestamp)
        end_index = np.searchsorted(world_timestamps['timestamp [ns]'], normalized_end_ts + base_timestamp)

        # Convert normalized timestamps to time in seconds
        start_time = normalized_start_ts / 1e9
        end_time = normalized_end_ts / 1e9
        duration = end_time - start_time

        # Check if the duration matches the duration of the gaze_data
        gaze_duration = (gaze_data['timestamp [ns]'].iloc[-1] - gaze_data['timestamp [ns]'].iloc[0]) / 1e9
        if abs(duration - gaze_duration) > 0.01:
            print(f"Warning: Duration mismatch between video segment and gaze data. Video duration: {duration}, Gaze data duration: {gaze_duration}")

        # Debug: Print calculated times
        # print(f"Event {int(event_id)}: {start_ts} to {end_ts}")
        # print(f"Start Time: {start_time}, End Time: {end_time}, Duration: {duration}")

        # Check if the start time and duration are valid
        if start_time < 0 or duration <= 0 or start_time + duration > video_duration:
            print(f"Invalid time range for {start_ts} to {end_ts}. Skipping.")
            continue

        # Output file path
        output_path = os.path.join(ad_save_dir, f'{int(event_id)}.mp4')

        # FFmpeg command to extract the video segment
        ffmpeg_command = [
            'ffmpeg',
            '-i', video,
            '-ss', f'{start_time:.3f}',  # Start time with millisecond precision
            '-t', f'{duration:.3f}',     # Duration with millisecond precision
            '-c:v', 'libx264',           # Video codec
            '-c:a', 'aac',               # Audio codec
            '-strict', 'experimental',
            '-y',                        # Overwrite output file
            output_path
        ]

        # Execute the FFmpeg command
        try:
            subprocess.run(ffmpeg_command, check=True, stderr=subprocess.PIPE)
            print(f"Segment saved: {output_path}")
        except subprocess.CalledProcessError as e:
            error_message = f"Error extracting segment {start_ts} to {end_ts}: {e.stderr.decode()}"
            print(error_message)
            log_error(error_message)

    print("Video processing completed.")

# Extracting Television AOI

In [10]:
def detect_tv_screen2(video_path, min_width, min_height, max_width, max_height, output_folder, save_frame):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Use GaussianBlur to reduce noise and improve edge detection
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Adaptive thresholding for better edge detection in varying lighting
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Loop over the contours
        for contour in contours:
            # Approximate the contour to reduce the number of points
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if it's a rectangle (has 4 points) and large enough
            if len(approx) == 4 and cv2.contourArea(approx) > 3000:
                x, y, w, h = cv2.boundingRect(approx)

                # Check if the rectangle is within the specified size range
                if (min_width <= w <= max_width) and (min_height <= h <= max_height):
                    # Calculate aspect ratio
                    aspect_ratio = float(w) / h
                    # Check if the aspect ratio is similar to a TV screen
                    if 1.33 < aspect_ratio < 1.78:  # Typical aspect ratios for TV screens
                        # Draw the contour on the frame
                        cv2.drawContours(frame, [approx], -1, (0, 255, 0), 3)

                        # Optionally save frames with drawn rectangles
                        if save_frame:
                            cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)  # Save each frame to a file
                        
                        # Save rectangle data
                        rectangle_data.append({
                            'frame': frame_index,
                            'x': x,
                            'y': y,
                            'width': w,
                            'height': h
                        })

        frame_index += 1  # Increment frame index

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):   
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage


In [ ]:
# testing
short_video = os.path.join(subj_folder, 'ads', '7.mp4')
output_file = os.path.join(subj_folder, 'ads', '7-rak')

detect_tv_screen2(short_video, min_width=400, min_height=200, max_width=600, max_height=400, output_folder=output_file, save_frame=True)

In [11]:
def filter_rectangles(csv_path, output_csv_path):
    # Load rectangle data
    rectangles = pd.read_csv(csv_path)

    # Calculate the aspect ratio (width/height)
    rectangles['aspect_ratio'] = rectangles['width'] / rectangles['height']

    # Define acceptable aspect ratio range (e.g., between 4/3 and 16/9)
    min_aspect_ratio = 4/3
    max_aspect_ratio = 16/9

    # Filter rectangles based on aspect ratio
    filtered_rectangles = rectangles[
        (rectangles['aspect_ratio'] >= min_aspect_ratio) &
        (rectangles['aspect_ratio'] <= max_aspect_ratio)
    ]

    # Save the filtered data to a new CSV file
    filtered_rectangles.to_csv(output_csv_path, index=False)
    print(f"Filtered rectangles saved to {output_csv_path}")

# # Define paths
# input_csv_path = os.path.join(output_file, 'rectangle_data.csv')  # Update this path
# output_csv_path = os.path.join(output_file, 'filtered_rectangle_data.csv')

# Call the function
# filter_rectangles(input_csv_path, output_csv_path)

In [5]:
# testing    
save_dir_2 = os.path.join(subj_folder, 'ads', '2-bk', 'v2')
save_dir_7 = os.path.join(subj_folder, 'ads', '7-rak')

input_csv_path = os.path.join(save_dir_2, 'rectangle_data.csv')
output_csv_path = os.path.join(save_dir_2, 'filtered_rectangle_data.csv')
filter_rectangles(input_csv_path, output_csv_path)

## Alternate TV Processing Functions

In [ ]:
def detect_tv_screen(video_path, min_width, min_height, max_width, max_height, output_folder, save_frame):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Edge detection
        edges = cv2.Canny(gray, 85, 255)

        # Find contours
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Loop over the contours
        for contour in contours:
            # Approximate the contour to reduce the number of points
            epsilon = 0.05 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if it's a rectangle (has 4 points) and large enough
            if len(approx) == 4 and cv2.contourArea(approx) > 3000:
                x, y, w, h = cv2.boundingRect(approx)

                # Check if the rectangle is within the specified size range
                if (min_width <= w <= max_width) and (min_height <= h <= max_height):
                    # Draw the contour on the frame
                    cv2.drawContours(frame, [approx], -1, (0, 255, 0), 3)

                    # Optionally save frames with drawn rectangles
                    if save_frame == True:
                        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)  # Save each frame to a file
                    
                    # Save rectangle data
                    rectangle_data.append({
                        'frame': frame_index,
                        'x': x,
                        'y': y,
                        'width': w,
                        'height': h
                    })

        frame_index += 1  # Increment frame index

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):   
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage: Only find screens between 50x50 and 200x150 pixels


In [ ]:
output_file = os.path.join(subj_folder, 'ads', '2-bk', 'v1')

detect_tv_screen(short_video, min_width=400, min_height=200, max_width=600, max_height=400, output_folder=output_file, save_frame=True)

1. Machine Learning-based Object Detection
a. Deep Learning Models
Utilize deep learning models like YOLO (You Only Look Once), SSD (Single Shot Multibox Detector), or Faster R-CNN to detect objects, including TV screens. These models are trained on large datasets and can generalize well to different environments.

YOLO (You Only Look Once): Fast and efficient for real-time detection.
SSD (Single Shot MultiBox Detector): Balances speed and accuracy well.
Faster R-CNN: Offers high accuracy but may be slower than YOLO and SSD.
Steps:
Data Preparation: Annotate a dataset of TV screens if a suitable pre-trained model isn't available.
Model Training: Train the model on the dataset (if using a custom dataset).
Inference: Use the trained model to detect TV screens in each video frame.
Output AOI: Save the detected bounding boxes as areas of interest (AOIs).
Libraries:
TensorFlow: Object Detection API
PyTorch: TorchVision
OpenCV with DNN Module: DNN Module Documentation

In [ ]:
def detect_tv_screen_yolo(video_path, confidence_threshold, output_folder):
    # Load YOLO
    # Load YOLO
    net = cv2.dnn.readNetFromDarknet("/Users/ebeard/Documents/darknet/cfg/yolov3.cfg", "/Users/ebeard/Documents/darknet/yolov3.weights")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Load class names
    with open("/Users/ebeard/Documents/darknet/data/coco.names", 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, channels = frame.shape

        # Prepare the image for YOLO
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Process detections
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                
                if confidence > confidence_threshold and classes[class_id] == 'tvmonitor':
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # Draw the rectangle
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    rectangle_data.append({
                        'frame': frame_index,
                        'x': x,
                        'y': y,
                        'width': w,
                        'height': h
                    })

        frame_index += 1

        # Save the frame with the detected screen
        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)
        
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage
# detect_tv_screen_yolo('video.mp4', 'yolov3.cfg', 'yolov3.weights', 'coco.names', 0.5, 'output_folder')


In [ ]:
output_file = os.path.join(subj_folder, 'ads', '2-bk', 'v3')
detect_tv_screen_yolo(short_video, 0.5, output_file)

3. Background Subtraction
If the TV screen is the only moving object in the scene or has a distinct movement pattern, background subtraction can be an effective approach.

Background Subtraction: Use methods like MOG2 or KNN from OpenCV to segment moving objects from the background.
Contour Detection: Apply contour detection on the foreground mask to identify the TV screen.

In [ ]:
def detect_tv_screen_background_subtraction(video_path, output_folder):
    # Create Background Subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()

    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Apply background subtraction
        fgMask = backSub.apply(frame)

        # Find contours in the mask
        contours, _ = cv2.findContours(fgMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 3000:  # Filter small areas
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                rectangle_data.append({
                    'frame': frame_index,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h
                })

        frame_index += 1

        # Save the frame with the detected screen
        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)

        # Display the resulting frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage
# detect_tv_screen_background_subtraction('video.mp4', 'output_folder')


In [ ]:
output_folder = os.path.join(subj_folder, 'ads', '2-bk', 'v4')
detect_tv_screen_background_subtraction(short_video, output_folder)

4. Color-Based Segmentation
If the TV screen has a distinct color profile (e.g., a blue tint from a video game or a specific show), you can use color-based segmentation to isolate it.

Color Space Conversion: Convert the frame to the HSV or LAB color space.
Color Thresholding: Apply thresholding to segment regions matching the screen's color.

In [ ]:
def detect_tv_screen_color_segmentation(video_path, lower_color, upper_color, output_folder):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create a mask based on the specified color range
        mask = cv2.inRange(hsv, lower_color, upper_color)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 3000:  # Filter small areas
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                rectangle_data.append({
                    'frame': frame_index,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h
                })

        frame_index += 1

        # Save the frame with the detected screen
        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)

        # Display the resulting frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage
# detect_tv_screen_color_segmentation('video.mp4', (100, 150, 150), (140, 255, 255), 'output_folder')


In [ ]:
output_folder = os.path.join(subj_folder, 'ads', '2-bk', 'v5')
detect_tv_screen_color_segmentation(short_video, (100, 150, 150), (140, 255, 255), output_folder)

5. Optical Flow Analysis
If the TV screen's content changes noticeably while the surrounding environment remains static, optical flow can be used to track motion differences between frames.

Optical Flow Calculation: Use methods like Lucas-Kanade or Farneback to compute the flow of moving regions.
Motion Segmentation: Identify areas with consistent motion patterns that match the TV screen.

Explanation of Key Steps:
Capture Video Frames: Open the video file and read the first frame to initialize the process.
Grayscale Conversion: Convert each frame to grayscale, as optical flow calculations require single-channel images.
Optical Flow Calculation: Use cv2.calcOpticalFlowFarneback to calculate the optical flow between the current frame and the previous frame.
Magnitude and Angle: Convert flow vectors to magnitude and angle using cv2.cartToPolar.
Thresholding: Apply a threshold to the magnitude to isolate regions of significant motion.
Contour Detection: Find contours in the binary mask created by the thresholding step.
Draw and Save Detected Regions: Draw bounding rectangles around detected contours, save the frame, and store AOI data.
Considerations:
Thresholding: The threshold value for motion magnitude needs to be adjusted based on the video content. A higher threshold may reduce false positives but might also miss more subtle motion.
Contour Area: Adjust the area threshold in cv2.contourArea(contour) > 5000 to filter out noise and only focus on larger moving objects, like a TV screen.
Performance: Optical flow calculations can be computationally expensive. Consider downscaling frames for faster processing if necessary.
Lighting Conditions: Optical flow can be sensitive to lighting changes, so ensure consistent lighting in your video.
Potential Improvements:
Adaptive Thresholding: Implement adaptive thresholding techniques to adjust the motion threshold dynamically based on frame content.
Post-processing: Use morphological operations to clean up the binary mask before contour detection.
ROI Refinement: Implement additional checks to refine detected ROIs based on expected size, shape, and position.

In [ ]:
def detect_tv_screen_optical_flow(video_path, threshold, output_folder):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()

    if not ret:
        print("Failed to read the video file.")
        return
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    frame_index = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)

    # Prepare to save rectangle data
    rectangle_data = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            pyr_scale=0.5, levels=3, winsize=15, 
                                            iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

        # Compute the magnitude and angle of the flow vectors
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Threshold the magnitude to create a binary mask
        mask = np.zeros_like(gray)
        mask[magnitude > threshold] = 255

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 5000:  # Adjust this threshold based on your needs
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Save rectangle data
                rectangle_data.append({
                    'frame': frame_index,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h
                })

        # Save the frame with the detected screen
        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)
        
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Update the previous frame and previous gray frame
        prev_gray = gray
        frame_index += 1

    cap.release()
    cv2.destroyAllWindows()

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage
# detect_tv_screen_optical_flow('video.mp4', threshold=5, output_folder='output_folder')


In [ ]:
output_folder = os.path.join(subj_folder, 'ads', '2-bk', 'v6')
detect_tv_screen_optical_flow(short_video, threshold=5, output_folder=output_folder)

# Average Gaze Data for each video frame

In [37]:
# missing synchronized data function
def average_gaze_data(gaze_data_path, world_timestamps_path):

    if not isinstance(gaze_data_path, pd.DataFrame):
        # Load gaze data
        gaze_data = pd.read_csv(gaze_data_path)
    else:
        try:
            gaze_data = gaze_data_path
        except Exception as e:
            print("Error loading gaze_data:", e)
            return None
        
    if not isinstance(world_timestamps_path, pd.DataFrame):
        # Load world timestamps
        world_timestamps = pd.read_csv(world_timestamps_path)
    else:
        try:
            world_timestamps = world_timestamps_path
        except Exception as e:
            print("Error loading world_timestamps:", e)
            return None
    
    # Calculate average frame duration in nanoseconds
    frame_durations = world_timestamps['timestamp [ns]'].diff().dropna()
    average_frame_duration = frame_durations.mean()

    # Calculate the index of the world timestamp for each gaze point
    gaze_data['world_timestamp_index'] = ((gaze_data['timestamp [ns]'] - world_timestamps['timestamp [ns]'].iloc[0]) / average_frame_duration).astype(int)
    gaze_data['world_timestamp_index'] = gaze_data['world_timestamp_index'].clip(0, len(world_timestamps) - 1)  # Prevent out of bounds indexing

    # Map the calculated index to the actual world timestamps
    gaze_data['world_timestamp [ns]'] = gaze_data['world_timestamp_index'].map(world_timestamps['timestamp [ns]'])

    # Group by the world timestamp and average the gaze coordinates
    averaged_gaze = gaze_data.groupby('world_timestamp [ns]').agg({
        'gaze x [px]': 'mean',
        'gaze y [px]': 'mean'
    }).reset_index()

    return averaged_gaze

# gaze_data = os.path.join(subj_folder, 'gaze.csv')
# world_timestamps = os.path.join(subj_folder, 'world_timestamps.csv')

# # Test the function with the previously loaded gaze_data and world_timestamps
# averaged_gaze_output = average_gaze_data(gaze_data, world_timestamps)

# averaged_gaze_output.to_csv(os.path.join(output_file,'averaged_gaze_data.csv'))

In [29]:
# testing


gaze_data = os.path.join(subj_folder, 'ads', f'event-{int(7)}_gaze.csv')

averaged_gaze_output = average_gaze_data(gaze_data, world_timestamps)
averaged_gaze_output.to_csv(os.path.join(save_dir_7, 'averaged_gaze_data.csv'))


# AOI For Each Frame

In [43]:
def interpolate_aoi_data_from_video(rectangle_df, video_path):
    """
    Interpolates missing AOI data (x, y, width, height) based on the number of frames in the video.
    
    Parameters:
    - rectangle_df (pd.DataFrame): A dataframe containing 'frame', 'x', 'y', 'width', 'height' columns for detected AOIs.
    - video_path (str): Path to the video file to get the total number of frames.
    
    Returns:
    - interpolated_df (pd.DataFrame): A dataframe with interpolated AOI values for each frame in the video.
    """

    # Check if rectangle_df is a file path
    if isinstance(rectangle_df, str):
        rectangle_df = pd.read_csv(rectangle_df)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    
    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()  # Close the video capture

    # Ensure rectangle_df contains only necessary columns and is sorted by frame
    rectangle_df = rectangle_df[['frame', 'x', 'y', 'width', 'height']].sort_values(by='frame').reset_index(drop=True)
    
    # Create a DataFrame that has all frame numbers from 0 to total_frames - 1
    all_frames = pd.DataFrame({'frame': np.arange(total_frames)})
    
    # Merge with the existing AOI data (this will insert NaNs for missing AOI data)
    full_frame_df = pd.merge(all_frames, rectangle_df, how='left', on='frame')
    
    # Interpolate missing values for x, y, width, height
    full_frame_df[['x', 'y', 'width', 'height']] = full_frame_df[['x', 'y', 'width', 'height']].interpolate(
        method='linear', limit_direction='forward', axis=0)
    
    # Fill any remaining NaNs with the nearest available values
    full_frame_df = full_frame_df.bfill()
    full_frame_df = full_frame_df.ffill()
    
    return full_frame_df

# Example usage:
# Assuming you have a filtered AOI dataframe 'filtered_rectangle_df' and video 'input_video.mp4'
# interpolated_aoi_df = interpolate_aoi_data_from_video(filtered_rectangle_df, 'input_video.mp4')

In [10]:
# testing
aoi_df = interpolate_aoi_data_from_video(filtered_rectangle_data, ad_video)

/var/folders/4z/r7mkx_mj2d73zrgl4w0848ww0000gq/T/ipykernel_64518/1299098279.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  full_frame_df.fillna(method='bfill', inplace=True)
/var/folders/4z/r7mkx_mj2d73zrgl4w0848ww0000gq/T/ipykernel_64518/1299098279.py:43: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  full_frame_df.fillna(method='ffill', inplace=True)


In [ ]:
def overlay_aoi_on_video(video_path, aoi_data, output_path):
    """
    Overlays AOI rectangles onto each frame of a video.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - aoi_data (pd.DataFrame): DataFrame containing AOI data with 'frame', 'x', 'y', 'width', 'height'.
    - output_path (str): Path to save the output video with AOIs overlaid.
    
    Returns:
    - None
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the codec and create VideoWriter object to save output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    # Loop through each frame in the video
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Get AOI for the current frame
        aoi_row = aoi_data[aoi_data['frame'] == i]
        
        if not aoi_row.empty:
            # Extract x, y, width, height of the AOI
            x = int(aoi_row['x'].values[0])
            y = int(aoi_row['y'].values[0])
            width = int(aoi_row['width'].values[0])
            height = int(aoi_row['height'].values[0])
            
            # Draw rectangle on the frame (Green color, thickness 2)
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        
        # Write the frame with the AOI overlay to the output video
        out.write(frame)
    
    # Release everything
    cap.release()
    out.release()
    print(f"Output video saved as {output_path}")

# Example usage:
# Assuming you have a video file 'input_video.mp4' and interpolated AOI DataFrame 'interpolated_aoi_df'
# overlay_aoi_on_video('input_video.mp4', interpolated_aoi_df, 'output_with_aoi.mp4')


In [ ]:
# testing
overlay_aoi_on_video(ad_video, aoi_df, os.path.join(ad_folder, 'output_with_aoi.mp4'))

# Map Gaze Onto Scene Video, Gaze Within AOI

In [47]:
def calculate_fixation_durations(fixation_data):
    """ Calculate fixation duration based on the number of gaze points for each fixation ID. """
    fixation_data = fixation_data.copy()
    fixation_data.loc[:,'fixation_duration'] = fixation_data.groupby('fixation id')['fixation id'].transform('count')
    return fixation_data

def is_point_in_aoi(gaze_x, gaze_y, aoi_row):
    """ Check if a given gaze point (gaze_x, gaze_y) is inside the AOI rectangle. """
    aoi_x = int(aoi_row['x'])
    aoi_y = int(aoi_row['y'])
    aoi_width = int(aoi_row['width'])
    aoi_height = int(aoi_row['height'])
    
    return aoi_x <= gaze_x <= aoi_x + aoi_width and aoi_y <= gaze_y <= aoi_y + aoi_height

def add_within_aoi_column(gaze_data, aoi_data, video_start_timestamp, frame_duration_ns):
    """ Add a 'within_aoi' column to the gaze data, indicating if the gaze point is within the AOI. """
    within_aoi = []
    
    for _, gaze_row in gaze_data.iterrows():
        gaze_timestamp = gaze_row['timestamp [ns]']
        gaze_x = int(gaze_row['gaze x [px]'])
        gaze_y = int(gaze_row['gaze y [px]'])
        
        # Find the corresponding frame based on the timestamp
        frame_idx = int((gaze_timestamp - video_start_timestamp) // frame_duration_ns)
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        
        if not aoi_row.empty:
            # Check if the gaze point is within the AOI
            within_aoi.append(is_point_in_aoi(gaze_x, gaze_y, aoi_row.iloc[0]))
        else:
            within_aoi.append(False)  # If no AOI for the frame, assume False
    
    gaze_data['within_aoi'] = within_aoi
    return gaze_data

def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays averaged gaze points (both fixation and non-fixation) onto each frame of a video.
    Scanpaths are drawn between averaged gaze points for continuous movement visualization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, gaze points, and scanpaths overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Calculate fixation durations based on the number of gaze points per fixation ID
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    fixation_data = calculate_fixation_durations(fixation_data)  # Calculate fixation durations
    
    # Calculate the frame duration based on average_rate
    input_container = av.open(video_path)
    input_stream = input_container.streams.video[0]
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Add the 'within_aoi' column to the gaze data
    gaze_data = add_within_aoi_column(gaze_data, aoi_data, video_start_timestamp, frame_duration_ns)

    # Define the path for the output video
    new_video_path = os.path.join(output_path, 'output_with_aoi_and_gaze.mp4')
    
    # Open the video file using PyAV
    output_container = av.open(new_video_path, mode='w')
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'

    # Variables to store previous averaged gaze point for scanpath drawing
    prev_gaze_x = None
    prev_gaze_y = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Find gaze points for the current frame and calculate the average
        gaze_points = gaze_data[(gaze_data['timestamp [ns]'] >= current_frame_timestamp) &
                                (gaze_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        
        if not gaze_points.empty:
            # Average the gaze points for this frame
            avg_gaze_x = int(gaze_points['gaze x [px]'].mean())
            avg_gaze_y = int(gaze_points['gaze y [px]'].mean())
            is_within_aoi = gaze_points['within_aoi'].mean() > 0.5  # If majority of points are inside AOI
            
            # Draw gaze point based on whether it's within AOI
            color = "blue" if is_within_aoi else "orange"
            draw.ellipse([(avg_gaze_x - 3, avg_gaze_y - 3), (avg_gaze_x + 3, avg_gaze_y + 3)], fill=color, outline=color)
            
            # Draw scanpath (line between consecutive averaged gaze points)
            if prev_gaze_x is not None and prev_gaze_y is not None:
                draw.line([(prev_gaze_x, prev_gaze_y), (avg_gaze_x, avg_gaze_y)], fill="blue", width=2)

            # Update the previous gaze point for the next frame
            prev_gaze_x, prev_gaze_y = avg_gaze_x, avg_gaze_y

        # Find fixation points within the current frame's time window and calculate the average
        fixation_rows = fixation_data[(fixation_data['timestamp [ns]'] >= current_frame_timestamp) &
                                      (fixation_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        if not fixation_rows.empty:
            # Average the fixation points for this frame
            avg_fixation_x = int(fixation_rows['gaze x [px]'].mean())
            avg_fixation_y = int(fixation_rows['gaze y [px]'].mean())
            avg_fixation_duration = fixation_rows['fixation_duration'].mean()
            
            # Base size of the circle and scaling it by the averaged fixation duration
            base_size = 3
            scaled_size = base_size + 2 * (avg_fixation_duration ** 0.5)
            radius = int(scaled_size)
            
            # Draw the averaged fixation point with no fill and size proportional to the averaged fixation duration
            draw.ellipse([(avg_fixation_x - radius, avg_fixation_y - radius), (avg_fixation_x + radius, avg_fixation_y + radius)], outline="blue", width=2)

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with averaged gaze points, fixations, and scanpath saved as {new_video_path}")

     # print the updated gaze_data with 'within_aoi' column
    # Save the updated gaze data to a new CSV file
    gaze_data.to_csv(os.path.join(output_path, 'updated_gaze_data.csv'), index=False)

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_averaged_gaze_and_scanpath.mp4', video_start_timestamp)


In [24]:
# test
# Assuming 'input_video.mp4' is the video, 'gaze_data' is the DataFrame for gaze, and 'aoi_data' is the AOI data
aligned_events = pd.read_csv(os.path.join(subj_folder, 'ads','aligned_events.csv'))
video_start_timestamp = aligned_events.loc[aligned_events['event_id'] == 2, 'nearest_timestamp_ns'].values[0]
overlay_gaze_and_aoi_on_video_ffmpeg(ad_video, gaze_data_raw, aoi_df, ad_folder, video_start_timestamp)


/var/folders/4z/r7mkx_mj2d73zrgl4w0848ww0000gq/T/ipykernel_64518/727462044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixation_data['fixation_duration'] = fixation_data.groupby('fixation id')['fixation id'].transform('count')
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 600 more times)


Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-019_tv-lego-42b0cb7d/ads/2/output_with_aoi_and_gaze.mp4


# Loop Through All Subjects

In [46]:
# with this loop we'll have: TV AOIs, and averaged gaze data for each frame.
for subj in subj_folders[:]:
    try:
        subj_id = subj.split('/')[5].split('_')[0]
        print('Splitting ads for ', subj_id)
        
        # subj info and files
        subj_folder = subj

        # if subj is in control condition or youtube condition, skip
        cond = subj.split('/')[5].split('_')[1].split('-')[1]

        if cond == 'control' or cond == 'youtube':
            print('Skipping control or youtube condition')
            continue


        try:
            video = glob.glob(os.path.join(subj, '*.mp4'))[0]
        except IndexError as e:
            error_message = f"Video file not found: {e}"
            print(error_message)
            log_error(error_message, subj=subj)
            continue

        try:
            events_df = pd.read_csv(os.path.join(subj_folder, 'events.csv'))
            world_timestamps_df = pd.read_csv(os.path.join(subj_folder, 'world_timestamps.csv'))
            task_events_df = pd.read_csv(os.path.join(t7sheild, 'eeg', f'{subj_id}_task-events.csv'))
            gaze = pd.read_csv(os.path.join(subj_folder, 'gaze.csv'))
            
        except FileNotFoundError as e:
            error_message = f"CSV file not found: {e}"
            print(error_message)
            log_error(error_message, subj=subj_id)
            continue

        save_dir = os.path.join(subj, 'ads')
        os.makedirs(save_dir, exist_ok=True)

        # align pupil labs with openbci data
        try:
            aligned = align_events(events_df, world_timestamps_df, task_events_df)
            aligned.to_csv(os.path.join(save_dir, 'aligned_events.csv'), index=False)
        
        except Exception as e:
            error_message = f"Error aligning events: {e}"
            print(error_message)
            log_error(error_message, subj=subj_id)
            continue

        # save open qc plots
        try:
            plot_adjusted_onsets_vs_world_timestamps(aligned, world_timestamps_df, save_dir)
            plot_nearest_timestamps(aligned, save_dir)
            plot_event_durations(aligned, save_dir)
        
        except Exception as e:
            error_message = f"Error generating plots: {e}"
            print(error_message)
            log_error(error_message, subj=subj_id)
            continue

        # extract pupil labs videos
        try:
            process_video_segments(video, world_timestamps_df, aligned, gaze, save_dir)
        except Exception as e:
            error_message = f"Error processing video segments: {e}"
            print(error_message)
            log_error(error_message, subj=subj_id)
            continue

        # extract telvision AOI
        ads_folders = glob.glob(os.path.join(subj, 'ads', '[0-9]*')) #'[0-9]*'))

        for ad_folder in ads_folders:
            short_video = glob.glob(os.path.join(ad_folder, '*.mp4'))[0]

            # detect tv screens
            try:
                detect_tv_screen2(short_video, min_width=400, min_height=200, max_width=600, max_height=400, output_folder=ad_folder, save_frame=True)
            
            except Exception as e:
                error_message = f"Detecting tv screens: {e}"
                print(error_message)
                log_error(error_message, subj=short_video)
                continue

            # filter rectangles
            try:

                csv_path = os.path.join(ad_folder, 'rectangle_data.csv')
                output_csv_path = os.path.join(ad_folder, 'filtered_rectangle_data.csv')
                filter_rectangles(csv_path, output_csv_path)
            
            except Exception as e:
                error_message = f"Error filtering rectangles: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            # average gaze data
            try:

                gaze_data_path = os.path.join(ad_folder, 'gaze.csv')
                averaged_gaze_output = average_gaze_data(gaze_data_path, world_timestamps_path=world_timestamps_df)
                averaged_gaze_output.to_csv(os.path.join(ad_folder, 'averaged_gaze_data.csv'))
            
            except Exception as e:
                error_message = f"Error averaging gaze data: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            # proces AOI data
            try:
                process_aoi_data(
                    aois_csv_path=os.path.join(ad_folder, 'filtered_rectangle_data.csv'),
                    timestamps_csv_path=os.path.join(ad_folder, 'averaged_gaze_data.csv'),
                    output_folder=ad_folder
                )
            except Exception as e:
                error_message = f"Error processing AOI data: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue
            


    except Exception as e:
        error_message = f"Unexpected error: {e}"
        print(error_message)
        log_error(error_message, subj=subj_id)

Splitting ads for  sub-021
Error aligning events: index 0 is out of bounds for axis 0 with size 0
Splitting ads for  sub-022
9
23634
15023
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/2/2.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/3/3.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/4/4.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/5/5.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/7/7.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/8/8.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/9/9.mp4
Segment saved: /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/10/10.mp4
Video processing completed.
Filtered rectangles saved to /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/2/filtered_rectangle_data.csv
Processed AOI data saved to /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/2
Filtered rectangle

In [48]:
# with this loop we'll have an AOI for each frame of the ad video, the pupil labs video with the AOI and gaze overlayed, and whether the gaze point is within the AOI or not.
for subj in subj_folders[1:]:
    try:
        subj_id = subj.split('/')[5].split('_')[0]
        print('Processing gaze data in ads for ', subj_id)
        
        # subj info and files
        subj_folder = subj

        # if subj is in control condition or youtube condition, skip
        cond = subj.split('/')[5].split('_')[1].split('-')[1]

        if cond == 'control' or cond == 'youtube':
            print('Skipping control or youtube condition')
            continue
        
        aligned_events = pd.read_csv(os.path.join(subj_folder, 'ads','aligned_events.csv'))
        ads_folders = glob.glob(os.path.join(subj, 'ads', '[0-9]*')) #'[0-9]*'))

        # print(aligned_events)

        for ad_folder in ads_folders:
            ad_video = glob.glob(os.path.join(ad_folder, '*.mp4'))[0]
            event_id = int(ad_folder.split('/')[-1])
            print('Processing ad ', event_id, 'in ', ad_video)



            try:
                filtered_rectangle_data = os.path.join(ad_folder, 'filtered_rectangle_data.csv')            
                aoi_df = interpolate_aoi_data_from_video(filtered_rectangle_data, ad_video)
                aoi_df.to_csv(os.path.join(ad_folder, 'processed_aoi_for_each_frame.csv'))

            except Exception as e:
                error_message = f"Error processing AOI for each frame: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue

            try:
                gaze_data_raw = os.path.join(ad_folder, 'gaze.csv')
                video_start_timestamp = aligned_events.loc[aligned_events['event_id'] == event_id, 'nearest_timestamp_ns'].values[0]
                # print(f"Video start timestamp: {video_start_timestamp}")

                overlay_gaze_and_aoi_on_video_ffmpeg(ad_video, gaze_data_raw, aoi_df, ad_folder, video_start_timestamp)
            
            except Exception as e:
                error_message = f"Error overlaying gaze to video: {e}"
                print(error_message)
                log_error(error_message, subj=subj_id)
                continue
            
    except Exception as e:
        error_message = f"Unexpected error: {e}"
        print(error_message)
        log_error(error_message, subj=subj_id)

Processing gaze data in ads for  sub-021
Unexpected error: [Errno 2] No such file or directory: '/Volumes/T7 Shield/fox/et/sub-021_tv-pickers-97606087/ads/aligned_events.csv'
Processing gaze data in ads for  sub-022
Processing ad  2 in  /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/2/2.mp4


No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 600 more times)


Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/2/output_with_aoi_and_gaze.mp4
Processing ad  3 in  /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/3/3.mp4


No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 132 more times)
No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 467 more times)


Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/3/output_with_aoi_and_gaze.mp4
Processing ad  4 in  /Volumes/T7 Shield/fox/et/sub-022_tv-cnn-6db56780/ads/4/4.mp4


...

Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-091_tv-foxnews-08834a6c/ads/8/output_with_aoi_and_gaze.mp4
Processing ad  9 in  /Volumes/T7 Shield/fox/et/sub-091_tv-foxnews-08834a6c/ads/9/9.mp4


No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 600 more times)


Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-091_tv-foxnews-08834a6c/ads/9/output_with_aoi_and_gaze.mp4
Processing ad  10 in  /Volumes/T7 Shield/fox/et/sub-091_tv-foxnews-08834a6c/ads/10/10.mp4


No accelerated colorspace conversion found from yuv420p to rgb24.
No accelerated colorspace conversion found from yuv420p to rgb24.
 (repeated 300 more times)


Output video with averaged gaze points, fixations, and scanpath saved as /Volumes/T7 Shield/fox/et/sub-091_tv-foxnews-08834a6c/ads/10/output_with_aoi_and_gaze.mp4
Processing gaze data in ads for  sub-101
Skipping control or youtube condition
Processing gaze data in ads for  sub-102
Skipping control or youtube condition
Processing gaze data in ads for  sub-103
Skipping control or youtube condition
Processing gaze data in ads for  sub-112
Skipping control or youtube condition
Processing gaze data in ads for  sub-113
Skipping control or youtube condition
Processing gaze data in ads for  sub-114
Skipping control or youtube condition
Processing gaze data in ads for  sub-120
Unexpected error: [Errno 2] No such file or directory: '/Volumes/T7 Shield/fox/et/sub-120_tv-continental-11b3e28a/ads/aligned_events.csv'
Processing gaze data in ads for  sub-121
Unexpected error: [Errno 2] No such file or directory: '/Volumes/T7 Shield/fox/et/sub-121_tv-continental-a8dba278/ads/aligned_events.csv'
Proce

# Graveyard
Where are all our unscessful functions/steps go to die. Gone but not forgotten.

In [ ]:
def calculate_fixation_durations(fixation_data):
    """ Calculate fixation duration based on the number of gaze points for each fixation ID. """
    fixation_data['fixation_duration'] = fixation_data.groupby('fixation id')['fixation id'].transform('count')
    return fixation_data

def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays averaged gaze points (both fixation and non-fixation) onto each frame of a video.
    Scanpaths are drawn between averaged gaze points for continuous movement visualization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, gaze points, and scanpaths overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """
    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Calculate fixation durations based on the number of gaze points per fixation ID
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    fixation_data = calculate_fixation_durations(fixation_data)  # Calculate fixation durations
    
    # Open the video file using PyAV
    input_container = av.open(video_path)
    output_container = av.open(output_path, mode='w')
    
    # Get the video stream
    input_stream = input_container.streams.video[0]
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'
    
    # Calculate the frame duration based on average_rate
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Variables to store previous averaged gaze point for scanpath drawing
    prev_gaze_x = None
    prev_gaze_y = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Find gaze points for the current frame and calculate the average
        gaze_points = gaze_data[(gaze_data['timestamp [ns]'] >= current_frame_timestamp) &
                                (gaze_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        
        if not gaze_points.empty:
            # Average the gaze points for this frame
            avg_gaze_x = int(gaze_points['gaze x [px]'].mean())
            avg_gaze_y = int(gaze_points['gaze y [px]'].mean())
            
            # Draw the averaged gaze point
            draw.ellipse([(avg_gaze_x - 3, avg_gaze_y - 3), (avg_gaze_x + 3, avg_gaze_y + 3)], fill="blue", outline="blue")
            
            # Draw scanpath (line between consecutive averaged gaze points)
            if prev_gaze_x is not None and prev_gaze_y is not None:
                draw.line([(prev_gaze_x, prev_gaze_y), (avg_gaze_x, avg_gaze_y)], fill="blue", width=2)

            # Update the previous gaze point for the next frame
            prev_gaze_x, prev_gaze_y = avg_gaze_x, avg_gaze_y

        # Find fixation points within the current frame's time window and calculate the average
        fixation_rows = fixation_data[(fixation_data['timestamp [ns]'] >= current_frame_timestamp) &
                                      (fixation_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        if not fixation_rows.empty:
            # Average the fixation points for this frame
            avg_fixation_x = int(fixation_rows['gaze x [px]'].mean())
            avg_fixation_y = int(fixation_rows['gaze y [px]'].mean())
            avg_fixation_duration = fixation_rows['fixation_duration'].mean()
            
            # Base size of the circle and scaling it by the averaged fixation duration
            base_size = 5
            scaled_size = base_size + 2 * (avg_fixation_duration ** 0.5)
            radius = int(scaled_size)
            
            # Draw the averaged fixation point with no fill and size proportional to the averaged fixation duration
            draw.ellipse([(avg_fixation_x - radius, avg_fixation_y - radius), (avg_fixation_x + radius, avg_fixation_y + radius)], outline="red", width=2)

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with averaged gaze points, fixations, and scanpath saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_averaged_gaze_and_scanpath.mp4', video_start_timestamp)


In [ ]:
def smooth_fixation_durations(fixation_data, window_size=5):
    """ Apply smoothing to fixation durations to avoid abrupt size changes. """
    fixation_data['smoothed_duration'] = fixation_data['fixation id'].rolling(window=window_size, min_periods=1).mean()
    return fixation_data

def interpolate_gaze_data(gaze_data, video_timestamps):
    """ Interpolate gaze data to fill in gaps between video frames for smoother transitions. """
    interp_func_x = interp1d(gaze_data['timestamp [ns]'], gaze_data['gaze x [px]'], fill_value="extrapolate")
    interp_func_y = interp1d(gaze_data['timestamp [ns]'], gaze_data['gaze y [px]'], fill_value="extrapolate")
    interpolated_gaze_x = interp_func_x(video_timestamps)
    interpolated_gaze_y = interp_func_y(video_timestamps)
    return interpolated_gaze_x, interpolated_gaze_y

def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays gaze points (both fixation and non-fixation), with smoother transitions, onto each frame of a video.
    Scanpaths are drawn between all gaze points for continuous movement visualization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, gaze points, and scanpaths overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Smooth the fixation durations
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    fixation_data = smooth_fixation_durations(fixation_data)  # Apply smoothing
    
    # Open the video file using PyAV
    input_container = av.open(video_path)
    output_container = av.open(output_path, mode='w')
    
    # Get the video stream
    input_stream = input_container.streams.video[0]
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'
    
    # Calculate the frame duration based on average_rate
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Prepare interpolation over video timestamps
    video_timestamps = [video_start_timestamp + frame_idx * frame_duration_ns for frame_idx in range(input_stream.frames)]
    interpolated_gaze_x, interpolated_gaze_y = interpolate_gaze_data(gaze_data, video_timestamps)

    # Variables to store previous gaze point for scanpath drawing
    prev_gaze_x = None
    prev_gaze_y = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Interpolated gaze points for smoother transitions
        gaze_x = int(interpolated_gaze_x[frame_idx])
        gaze_y = int(interpolated_gaze_y[frame_idx])
        
        # Draw gaze point
        draw.ellipse([(gaze_x - 3, gaze_y - 3), (gaze_x + 3, gaze_y + 3)], fill="blue", outline="blue")

        # Draw scanpath (line between consecutive gaze points)
        if prev_gaze_x is not None and prev_gaze_y is not None:
            draw.line([(prev_gaze_x, prev_gaze_y), (gaze_x, gaze_y)], fill="blue", width=2)

        # Update the previous gaze point for the next frame
        prev_gaze_x, prev_gaze_y = gaze_x, gaze_y

        # Handle fixation points for this frame
        fixation_row = fixation_data[fixation_data['timestamp [ns]'] == current_frame_timestamp]
        if not fixation_row.empty:
            fixation_duration = fixation_row['smoothed_duration'].values[0]
            
            base_size = 5
            scaled_size = base_size + 2 * (fixation_duration ** 0.5)
            radius = int(scaled_size)
            
            # Draw the fixation point with no fill and size proportional to smoothed duration
            draw.ellipse([(gaze_x - radius, gaze_y - radius), (gaze_x + radius, gaze_y + radius)], outline="red", width=2)

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with gaze points, fixations, and scanpath saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_gaze_and_scanpath.mp4', video_start_timestamp)


In [ ]:
def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays fixation points (with reasonable sizes) and scanpath onto each frame of a video,
    using the fixation column from the gaze data, following Pupil Labs formatting.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, fixation points, and scanpaths overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)
        
    # Filter the gaze data to only include fixation points
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    
    # Open the video file using PyAV
    input_container = av.open(video_path)
    output_container = av.open(output_path, mode='w')
    
    # Get the video stream
    input_stream = input_container.streams.video[0]
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'
    
    # Calculate the frame duration based on average_rate
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)

    # Variables to store previous fixation point for scanpath drawing
    prev_fixation_x = None
    prev_fixation_y = None
    prev_fixation_id = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            # Draw a brighter green AOI (brighter RGB value for green)
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Find fixation points in the current frame's time window
        fixation_rows = fixation_data[(fixation_data['timestamp [ns]'] >= current_frame_timestamp) &
                                      (fixation_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]

        if not fixation_rows.empty:
            draw = ImageDraw.Draw(img)
            
            # Iterate over all fixation points in this frame
            for _, fixation_row in fixation_rows.iterrows():
                fixation_x = int(fixation_row['gaze x [px]'])
                fixation_y = int(fixation_row['gaze y [px]'])
                fixation_id = fixation_row['fixation id']

                # Calculate the fixation duration (number of points with the same fixation id)
                fixation_duration = len(fixation_data[fixation_data['fixation id'] == fixation_id])

                # Pupil Labs circle size - base size of 5, scale with moderate multiplier
                base_size = 5
                scaled_size = base_size + 2 * (fixation_duration ** 0.5)  # Moderate scaling based on duration
                radius = int(scaled_size)  # Ensure reasonable sizes
                
                # Draw the fixation point with no fill (transparent) and size proportional to fixation duration
                draw.ellipse([(fixation_x - radius, fixation_y - radius), (fixation_x + radius, fixation_y + radius)], outline="red", width=2)

                # Draw the scanpath (line between consecutive fixation points)
                if prev_fixation_x is not None and prev_fixation_y is not None and prev_fixation_id != fixation_id:
                    draw.line([(prev_fixation_x, prev_fixation_y), (fixation_x, fixation_y)], fill="blue", width=2)

                # Update the previous fixation point for the next frame
                prev_fixation_x, prev_fixation_y = fixation_x, fixation_y
                prev_fixation_id = fixation_id

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with fixation points and scanpath saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_fixations_and_scanpath.mp4', video_start_timestamp)




In [ ]:
def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays gaze points and AOI rectangles onto each frame of a video using PyAV for precise timestamp handling.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs and gaze points overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Open the video file using PyAV
    input_container = av.open(video_path)
    output_container = av.open(output_path, mode='w')
    
    # Get the video stream
    input_stream = input_container.streams.video[0]
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'
    
    # Calculate the frame duration based on average_rate
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline="green", width=2)
        
        # Find the closest gaze point based on the current frame timestamp (use raw gaze data)
        gaze_row = gaze_data[(gaze_data['timestamp [ns]'] >= current_frame_timestamp) &
                             (gaze_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        if not gaze_row.empty:
            gaze_x = int(gaze_row['gaze x [px]'].values[0])
            gaze_y = int(gaze_row['gaze y [px]'].values[0])
            draw.ellipse([(gaze_x - 5, gaze_y - 5), (gaze_x + 5, gaze_y + 5)], fill="red")

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with AOI and gaze data saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_raw_gaze.mp4', video_start_timestamp)

  

In [ ]:
def calculate_fixation_durations(fixation_data):
    """ Calculate fixation duration based on the number of gaze points for each fixation ID. """
    fixation_data['fixation_duration'] = fixation_data.groupby('fixation id')['fixation id'].transform('count')
    return fixation_data

def overlay_gaze_and_aoi_on_video_ffmpeg(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays gaze points (both fixation and non-fixation), with smoother transitions, onto each frame of a video.
    Scanpaths are drawn between all gaze points for continuous movement visualization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Raw gaze data with 'timestamp [ns]', 'gaze x [px]', 'gaze y [px]', 'fixation id' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs, gaze points, and scanpaths overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """
    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Check if aoi_data is a file path
    if isinstance(aoi_data, str):
        aoi_data = pd.read_csv(aoi_data)

    # Calculate fixation durations based on the number of gaze points per fixation ID
    fixation_data = gaze_data[gaze_data['fixation id'].notna()]
    fixation_data = calculate_fixation_durations(fixation_data)  # Calculate fixation durations
    
    # Open the video file using PyAV
    input_container = av.open(video_path)
    output_container = av.open(output_path, mode='w')
    
    # Get the video stream
    input_stream = input_container.streams.video[0]
    output_stream = output_container.add_stream('mpeg4', rate=input_stream.average_rate)
    output_stream.width = input_stream.width
    output_stream.height = input_stream.height
    output_stream.pix_fmt = 'yuv420p'
    
    # Calculate the frame duration based on average_rate
    frame_rate = input_stream.average_rate
    frame_duration_ns = int(1 / frame_rate * 1e9)
    
    # Variables to store previous gaze point for scanpath drawing
    prev_gaze_x = None
    prev_gaze_y = None
    
    # Iterate over frames in the input video
    for frame_idx, frame in enumerate(input_container.decode(video=0)):
        # Convert the PyAV frame to a PIL image for drawing
        img = frame.to_image()

        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns

        # Draw AOI for the current frame if available
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        if not aoi_row.empty:
            draw = ImageDraw.Draw(img)
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            draw.rectangle([(aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height)], outline=(0, 255, 0), width=4)

        # Find gaze points for the current frame
        gaze_points = gaze_data[(gaze_data['timestamp [ns]'] >= current_frame_timestamp) &
                                (gaze_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        
        if not gaze_points.empty:
            for _, gaze_row in gaze_points.iterrows():
                gaze_x = int(gaze_row['gaze x [px]'])
                gaze_y = int(gaze_row['gaze y [px]'])
                
                # Draw the gaze point
                draw.ellipse([(gaze_x - 3, gaze_y - 3), (gaze_x + 3, gaze_y + 3)], fill="blue", outline="blue")
                
                # Draw scanpath (line between consecutive gaze points)
                if prev_gaze_x is not None and prev_gaze_y is not None:
                    draw.line([(prev_gaze_x, prev_gaze_y), (gaze_x, gaze_y)], fill="blue", width=2)

                # Update the previous gaze point for the next frame
                prev_gaze_x, prev_gaze_y = gaze_x, gaze_y

        # Find fixation points within the current frame's time window
        fixation_rows = fixation_data[(fixation_data['timestamp [ns]'] >= current_frame_timestamp) &
                                      (fixation_data['timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        if not fixation_rows.empty:
            for _, fixation_row in fixation_rows.iterrows():
                fixation_x = int(fixation_row['gaze x [px]'])
                fixation_y = int(fixation_row['gaze y [px]'])
                fixation_duration = fixation_row['fixation_duration']
                
                # Base size of the circle and scaling it by the fixation duration
                base_size = 5
                scaled_size = base_size + 2 * (fixation_duration ** 0.5)
                radius = int(scaled_size)
                
                # Draw the fixation point with no fill and size proportional to fixation duration
                draw.ellipse([(fixation_x - radius, fixation_y - radius), (fixation_x + radius, fixation_y + radius)], outline="red", width=2)

        # Convert the image back to a PyAV frame
        frame = av.VideoFrame.from_image(img)

        # Encode and write the frame
        packet = output_stream.encode(frame)
        output_container.mux(packet)
    
    # Finalize the video
    packet = output_stream.encode(None)
    output_container.mux(packet)

    # Close the input and output containers
    input_container.close()
    output_container.close()

    print(f"Output video with gaze points, fixations, and scanpath saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video_ffmpeg('input_video.mp4', gaze_data_raw_df, aoi_df, 'output_with_gaze_and_scanpath.mp4', video_start_timestamp)


In [ ]:
def overlay_gaze_and_aoi_on_video(video_path, gaze_data, aoi_data, output_path, video_start_timestamp):
    """
    Overlays gaze points and AOI rectangles onto each frame of a video, adjusting for timestamp synchronization.
    
    Parameters:
    - video_path (str): Path to the input video file.
    - gaze_data (pd.DataFrame): Gaze data with 'world_timestamp [ns]', 'gaze x [px]', 'gaze y [px]' columns.
    - aoi_data (pd.DataFrame): AOI data with 'frame', 'x', 'y', 'width', 'height' columns for each frame.
    - output_path (str): Path to save the output video with AOIs and gaze points overlaid.
    - video_start_timestamp (int): The starting timestamp of the subset video in nanoseconds.
    
    Returns:
    - None
    """

    # Check if gaze_data is a file path
    if isinstance(gaze_data, str):
        gaze_data = pd.read_csv(gaze_data)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the codec and create VideoWriter object to save output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    # Calculate the time duration per frame (in nanoseconds)
    frame_duration_ns = int(1 / fps * 1e9)
    
    # Loop through each frame in the video
    for frame_idx in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate the timestamp for the current frame
        current_frame_timestamp = video_start_timestamp + frame_idx * frame_duration_ns
        
        # Get AOI for the current frame
        aoi_row = aoi_data[aoi_data['frame'] == frame_idx]
        
        if not aoi_row.empty:
            # Extract x, y, width, height of the AOI
            aoi_x = int(aoi_row['x'].values[0])
            aoi_y = int(aoi_row['y'].values[0])
            aoi_width = int(aoi_row['width'].values[0])
            aoi_height = int(aoi_row['height'].values[0])
            
            # Draw the AOI rectangle on the frame (Green color, thickness 2)
            cv2.rectangle(frame, (aoi_x, aoi_y), (aoi_x + aoi_width, aoi_y + aoi_height), (0, 255, 0), 2)
        
        # Find the closest gaze point based on the current frame timestamp
        gaze_row = gaze_data[(gaze_data['world_timestamp [ns]'] >= current_frame_timestamp) & 
                             (gaze_data['world_timestamp [ns]'] < current_frame_timestamp + frame_duration_ns)]
        
        if not gaze_row.empty:
            # Extract gaze coordinates (gaze_x and gaze_y)
            gaze_x = int(gaze_row['gaze x [px]'].values[0])
            gaze_y = int(gaze_row['gaze y [px]'].values[0])
            
            # Draw a circle representing the gaze point (Red color, thickness -1 for filled circle)
            cv2.circle(frame, (gaze_x, gaze_y), radius=10, color=(0, 0, 255), thickness=-1)
        
        # Write the frame with AOI and gaze overlay to the output video
        out.write(frame)
    
    # Release everything
    cap.release()
    out.release()
    print(f"Output video with AOI and gaze data saved as {output_path}")

# Example usage:
# video_start_timestamp = 1712761742852933376  # Taken from the 'aligned_events.csv'
# overlay_gaze_and_aoi_on_video('input_video.mp4', gaze_data, aoi_data, 'output_with_aoi_and_gaze.mp4', video_start_timestamp)


In [ ]:
def process_video_test(video_path, gaze_data_path, rectangle_data_path, output_video_path, output_folder):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        # Open the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video file")
            return
        
        # Prepare the video writer to save output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(3)), int(cap.get(4))))
        
        frame_index = 1  # Assuming the first frame is index 1
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                x, y, w, h = int(rect['x']), int(rect['y']), int(rect['width']), int(rect['height'])

                # Ensure the AOI doesn't go out of frame bounds
                x, y = max(0, x), max(0, y)
                w = min(w, frame.shape[1] - x)
                h = min(h, frame.shape[0] - y)

                # Debugging output
                print(f"Frame {frame_index}: AOI at ({x}, {y}) with width {w} and height {h}")

                # Draw rectangle (AOI)
                if np.all(np.isfinite([x, y, w, h])):
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:
                x, y, w, h = 0, 0, 0, 0  # No AOI for this frame
                print(f"Frame {frame_index}: No AOI found")

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Check if gaze is within the rectangle
                    if x <= gaze_x <= x + w and y <= gaze_y <= y + h:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 0, 255), -1)  # Red circle for gaze within AOI
                    else:
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (255, 0, 0), -1)  # Blue circle for gaze outside AOI

            # Add frame index text for debugging
            cv2.putText(frame, f'Frame: {frame_index}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Write the frame with overlays to the output video
            out.write(frame)
            
            frame_index += 1  # Increment frame index

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

        for i in range(2):
            cv2.waitKey(1)
 
    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure resources are released if an error occurs
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        for i in range(2):
            cv2.waitKey(1)


In [ ]:
def process_video_noScale(video_path, gaze_data_path, rectangle_data_path, output_video_path, output_folder):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        # Open the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video file")
            return
        
        # Prepare the video writer to save output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(3)), int(cap.get(4))))
        
        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                x, y, w, h = int(rect['x']), int(rect['y']), int(rect['width']), int(rect['height'])

                # Ensure the AOI doesn't go out of frame bounds
                x, y = max(0, x), max(0, y)
                w = min(w, frame.shape[1] - x)
                h = min(h, frame.shape[0] - y)

                # Draw rectangle (AOI)
                if np.all(np.isfinite([x, y, w, h])):
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:
                x, y, w, h = 0, 0, 0, 0  # No AOI for this frame

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Check if gaze is within the rectangle
                    if x <= gaze_x <= x + w and y <= gaze_y <= y + h:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 0, 255), -1)  # Red circle for gaze within AOI
                    else:
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (255, 0, 0), -1)  # Blue circle for gaze outside AOI

            # Add frame index text for debugging
            cv2.putText(frame, f'Frame: {frame_index}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Write the frame with overlays to the output video
            out.write(frame)
            
            frame_index += 1  # Increment frame index

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

        for i in range(2):
            cv2.waitKey(1)
 
    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure resources are released if an error occurs
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        for i in range(2):
            cv2.waitKey(1)


In [ ]:
def process_video_2(video_path, gaze_data_path, rectangle_data_path, output_video_path, output_folder, scale_factor=1.0):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        # Open the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video file")
            return
        
        # Prepare the video writer to save output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(3)), int(cap.get(4))))
        
        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                cx, cy = int(rect['x']) + int(rect['width']) // 2, int(rect['y']) + int(rect['height']) // 2
                nw, nh = int(rect['width'] * scale_factor), int(rect['height'] * scale_factor)
                nx, ny = cx - nw // 2, cy - nh // 2

                # Ensure the AOI doesn't go out of frame bounds
                nx, ny = max(0, nx), max(0, ny)
                nw = min(nw, frame.shape[1] - nx)
                nh = min(nh, frame.shape[0] - ny)

                # Draw scaled rectangle (AOI)
                if np.all(np.isfinite([nx, ny, nw, nh])):
                    cv2.rectangle(frame, (nx, ny), (nx + nw, ny + nh), (0, 255, 0), 2)
            else:
                nx, ny, nw, nh = 0, 0, 0, 0  # No AOI for this frame

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Check if gaze is within the scaled rectangle
                    if nx <= gaze_x <= nx + nw and ny <= gaze_y <= ny + nh:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 0, 255), -1)  # Red circle for gaze within AOI
                    else:
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (255, 0, 0), -1)  # Blue circle for gaze outside AOI

            # Add frame index text for debugging
            cv2.putText(frame, f'Frame: {frame_index}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Write the frame with overlays to the output video
            out.write(frame)
            
            frame_index += 1  # Increment frame index

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

        for i in range(2):
            cv2.waitKey(1)
 
    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure resources are released if an error occurs
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        for i in range(2):
            cv2.waitKey(1)


In [ ]:
def process_video(video_path, gaze_data_path, rectangle_data_path, output_video_path, output_folder, scale_factor=1.0):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        # Open the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video file")
            return
        
        # Prepare the video writer to save output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(3)), int(cap.get(4))))
        
        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                cx, cy = int(rect['x']) + int(rect['width']) // 2, int(rect['y']) + int(rect['height']) // 2
                nw, nh = int(rect['width'] * scale_factor), int(rect['height'] * scale_factor)
                nx, ny = cx - nw // 2, cy - nh // 2
                # Draw scaled rectangle (AOI)
                if np.all(np.isfinite([nx, ny, nw, nh])):
                    cv2.rectangle(frame, (nx, ny), (nx + nw, ny + nh), (0, 255, 0), 2)
            else:
                nx, ny, nw, nh = 0, 0, 0, 0  # No AOI for this frame

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Check if gaze is within the scaled rectangle
                    if nx <= gaze_x <= nx + nw and ny <= gaze_y <= ny + nh:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 0, 255), -1)  # Red circle for gaze within AOI
                    else:
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (255, 0, 0), -1)  # Blue circle for gaze outside AOI

            # Write the frame with overlays to the output video
            out.write(frame)
            
            frame_index += 1  # Increment frame index

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

        for i in range(2):
            cv2.waitKey(1)
 
    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure resources are released if an error occurs
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        for i in range(2):
            cv2.waitKey(1)

# Example of how to call the function
# gaze_path = os.path.join(output_file, 'averaged_gaze_data.csv')
# rectangle_path = os.path.join(output_file, 'processed_aoi_for_each_frame.csv')
# output_video = os.path.join(output_file, 'world_video_with_gaze_and_aoi.mp4')

# process_video(video, gaze_path, rectangle_path, output_video, output_folder=output_file)


In [ ]:
def draw_aoi_on_video(video_path, aoi_data_path, output_video_path):
    # Load the AOI data
    aoi_data = pd.read_csv(aoi_data_path)

    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Prepare the video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get the AOI for the current frame
        if frame_index < len(aoi_data):
            row = aoi_data.iloc[frame_index]
            x, y, w, h = int(row['x']), int(row['y']), int(row['width']), int(row['height'])

            # Draw the AOI rectangle on the frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Write the frame with the AOI drawn
        out.write(frame)

        frame_index += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video with AOI overlay saved to {output_video_path}")

# Example usage:
video_path = '/mnt/data/7.mp4'
aoi_data_path = '/mnt/data/complete_aoi_data.csv'
output_video_path = '/mnt/data/video_with_aoi.mp4'

draw_aoi_on_video(video_path, aoi_data_path, output_video_path)

In [ ]:
def ensure_aoi_for_all_frames(video_path, filtered_rectangle_data_path, output_folder):
    # Load the filtered AOI data
    aois_df = pd.read_csv(filtered_rectangle_data_path)

    # Open the video to get the total number of frames
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Create a DataFrame with all frame numbers
    all_frames_df = pd.DataFrame({'frame': range(total_frames)})

    # Merge the AOI data with the all frames DataFrame
    merged_df = pd.merge(all_frames_df, aois_df, on='frame', how='left')

    # Interpolate missing AOIs
    merged_df[['x', 'y', 'width', 'height']] = merged_df[['x', 'y', 'width', 'height']].interpolate(method='linear', limit_direction='both')

    # Fill any remaining NaN values (at the start or end of the video) using forward fill
    merged_df[['x', 'y', 'width', 'height']] = merged_df[['x', 'y', 'width', 'height']].fillna(method='ffill')
    merged_df[['x', 'y', 'width', 'height']] = merged_df[['x', 'y', 'width', 'height']].fillna(method='bfill')

    # Save the cleaned data to a new CSV file
    output_path = os.path.join(output_folder, 'complete_aoi_data.csv')
    merged_df.to_csv(output_path, index=False)

    print(f"Completed AOI data saved to {output_path}")

# Example usage:
filtered_rectangle_data_path = os.path.join(save_dir_7,'filtered_rectangle_data.csv')
output_folder = save_dir_7
short_video = os.path.join(subj_folder,'ads','7.mp4')

ensure_aoi_for_all_frames(short_video, filtered_rectangle_data_path, output_folder)

In [ ]:
def extract_aoi_templates(rectangle_data_path, video_path, output_folder):
    # Load rectangle data
    rectangle_data = pd.read_csv(rectangle_data_path)

    # Open the video
    cap = cv2.VideoCapture(video_path)

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Extract and save AOI templates
    for index, row in rectangle_data.iterrows():
        cap.set(cv2.CAP_PROP_POS_FRAMES, row['frame'])
        ret, frame = cap.read()
        if ret:
            x, y, w, h = int(row['x']), int(row['y']), int(row['width']), int(row['height'])
            aoi = frame[y:y+h, x:x+w]
            cv2.imwrite(f"{output_folder}/aoi_template_{row['frame']}.jpg", aoi)

    cap.release()

def match_missing_aois_with_templates(video_path, missing_frames, template_folder, output_folder, save_frame=True):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'matched_frames')
    os.makedirs(frame_folder, exist_ok=True)

    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Only process frames that are missing AOIs
        if frame_index in missing_frames:
            best_match = None
            best_val = None

            # Iterate over all templates and find the best match
            for template_file in os.listdir(template_folder):
                template_path = os.path.join(template_folder, template_file)
                template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                res = cv2.matchTemplate(frame_gray, template, cv2.TM_CCOEFF_NORMED)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

                if best_val is None or max_val > best_val:
                    best_val = max_val
                    best_match = (max_loc, template.shape[1], template.shape[0])

            # If a good match is found, save it
            if best_match:
                x, y = best_match[0]
                w, h = best_match[1], best_match[2]

                # Draw rectangle on the matched region
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Save rectangle data
                rectangle_data.append({
                    'frame': frame_index,
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h
                })

                # Optionally save frames with drawn rectangles
                if save_frame:
                    cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)  # Save each frame to a file

        frame_index += 1

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'matched_rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

    print(f"Matched AOI data saved to {output_folder}")

# Example usage:
# 1. Extract AOI templates from the first pass:
extract_aoi_templates(rectangle_data_path=os.path.join(save_dir_7,'rectangle_data.csv'), 
                      video_path=os.path.join(subj_folder,'ads','7.mp4'),
                      output_folder=os.path.join(save_dir_7,'aoi_templates'))

# 2. Match missing AOIs using the extracted templates:
missing_frames = find_missing_frames(video_path=os.path.join(subj_folder,'ads','7.mp4'), 
                                     rectangle_data_path=os.path.join(save_dir_7,'rectangle_data.csv'))
match_missing_aois_with_templates(video_path=short_video, 
                                  missing_frames=missing_frames, 
                                  template_folder=os.path.join(save_dir_7,'aoi_templates'),
                                  output_folder=os.path.join(save_dir_7,'matched_frames'))

In [ ]:
def find_missing_frames(video_path, rectangle_data_path):
    # Load rectangle data
    rectangle_data = pd.read_csv(rectangle_data_path)

    # Open the video to get the total number of frames
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Extract the frames that have AOIs from the CSV
    frames_with_aois = set(rectangle_data['frame'].values)

    # Find the missing frames
    all_frames = set(range(total_frames))
    missing_frames = list(all_frames - frames_with_aois)

    return missing_frames

# Example usage:
video_path = short_video
rectangle_data_path = os.path.join(save_dir_7,'rectangle_data.csv')

missing_frames = find_missing_frames(video_path, rectangle_data_path)
print(f"Missing frames: {missing_frames}")


In [ ]:
def process_aoi_data_interpolation(aois_csv_path, timestamps_csv_path, output_folder):
    # Load the AOIs data
    aois_df = pd.read_csv(aois_csv_path)

    # Load the frame timestamps data
    timestamps_df = pd.read_csv(timestamps_csv_path)

    # Assume the world timestamp data frames are indexed properly as frame numbers
    timestamps_df['frame'] = timestamps_df.index + 1  # Sequential frame numbers starting from 1

    # Merge the dataframes to align AOI with frames
    merged_df = pd.merge_asof(timestamps_df.sort_values('frame'), 
                              aois_df.sort_values('frame'),
                              on='frame', 
                              direction='nearest',
                              tolerance=5)  # Adjust tolerance if needed

    # Interpolate missing AOI values to ensure every frame has an AOI
    merged_df[['x', 'y', 'width', 'height']] = merged_df[['x', 'y', 'width', 'height']].interpolate(method='linear', limit_direction='both')

    # Handle any remaining NaN values by forward filling as a fallback
    merged_df[['x', 'y', 'width', 'height']] = merged_df[['x', 'y', 'width', 'height']].fillna(method='ffill')

    # Clean up the DataFrame for final output
    final_df = merged_df[['frame', 'x', 'y', 'width', 'height']]

    # Save the cleaned data to a new CSV file
    final_df.to_csv(os.path.join(output_folder, 'processed_aoi_for_each_frame.csv'), index=False)

    print(f"Processed AOI data saved to {output_folder}")

# Example usage
process_aoi_data_interpolation(
    aois_csv_path=os.path.join(save_dir_7, 'filtered_rectangle_data.csv'),
    timestamps_csv_path=os.path.join(save_dir_7,'averaged_gaze_data.csv'),
    output_folder=save_dir_7
)


In [ ]:
def process_aoi_data(aois_csv_path, timestamps_csv_path, output_folder, pos_diff_threshold=0.05, rolling_frame_count=3):
    # Load the AOIs data
    aois_df = pd.read_csv(aois_csv_path)

    # Load the frame timestamps data
    timestamps_df = pd.read_csv(timestamps_csv_path)

    # Assume the world timestamp data frames are indexed properly as frame numbers
    timestamps_df['frame'] = timestamps_df.index + 1  # Sequential frame numbers starting from 1

    # Map frames from AOIs to timestamps using nearest match (simplified merge without tolerance)
    merged_df = pd.merge_asof(timestamps_df.sort_values('frame'), 
                              aois_df.sort_values('frame'),
                              on='frame', 
                              direction='nearest')

    # Add previous frame AOI info for comparison
    merged_df['prev_x'] = merged_df['x'].shift(1)
    merged_df['prev_y'] = merged_df['y'].shift(1)

    # Calculate percentage differences for position only
    merged_df['diff_x'] = (merged_df['x'] - merged_df['prev_x']).abs() / merged_df['prev_x']
    merged_df['diff_y'] = (merged_df['y'] - merged_df['prev_y']).abs() / merged_df['prev_y']

    # Identify rows that exceed position difference threshold
    condition = (merged_df['diff_x'] > pos_diff_threshold) | (merged_df['diff_y'] > pos_diff_threshold)

    # Apply rolling average for the identified frames based on the specified number of frames
    for index, row in merged_df[condition].iterrows():
        window_start = max(index - (rolling_frame_count // 2), 0)
        window_end = min(index + (rolling_frame_count // 2), len(merged_df) - 1)
        window = merged_df.loc[window_start:window_end]
        merged_df.loc[index, ['x', 'y']] = window[['x', 'y']].mean().astype(int)

    # Clean up the DataFrame for final output
    final_df = merged_df[['frame', 'x', 'y', 'width', 'height']]

    # Save the cleaned data to a new CSV file
    final_df.to_csv(os.path.join(output_folder, 'processed_aoi_for_each_frame.csv'), index=False)

    print(f"Processed AOI data saved to {output_folder}")

# process_aoi_data(
#     aois_csv_path=os.path.join(output_file, 'filtered_rectangle_data.csv'),
#     timestamps_csv_path=os.path.join(subj_folder,'world_timestamps.csv'),
#     output_folder=output_file,
#     pos_diff_threshold=0.05,
#     rolling_frame_count=3
# )

In [ ]:
# template matching for tv screen detection
def detect_tv_screen_template(video_path, template_path, threshold, output_folder):
    # Load the template
    template = cv2.imread(template_path, 0)
    w, h = template.shape[::-1]

    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_index = 0  # To keep track of frame numbers

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_folder = os.path.join(output_folder, 'tv_frames')
    os.makedirs(frame_folder, exist_ok=True)
    
    # Prepare to save rectangle data
    rectangle_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply template matching
        res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)

        # Draw rectangle around matched region
        for pt in zip(*loc[::-1]):
            cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
            rectangle_data.append({
                'frame': frame_index,
                'x': pt[0],
                'y': pt[1],
                'width': w,
                'height': h
            })

        frame_index += 1

        # Save the frame with the detected screen
        cv2.imwrite(f'{frame_folder}/frame_{frame_index}.jpg', frame)

        # Display the resulting frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'rectangle_data.csv'), 'w', newline='') as csvfile:
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in rectangle_data:
            writer.writerow(data)

# Example usage
# detect_tv_screen_template('video.mp4', 'template.jpg', 0.8, 'output_folder')


In [ ]:
# function for finding TV screens in a video using YOLOv3
def detect_and_save_aoi(video_path, output_folder, aoi_object, skip_frames=5):

    if not aoi_object:
        print("Please provide an AOI object to detect.")
        return
    
    net = cv2.dnn.readNet("/Users/ebeard/Documents/darknet/yolov3.weights", "/Users/ebeard/Documents/darknet/cfg/yolov3.cfg")
    layer_names = net.getLayerNames()
    classes = []
    with open("/Users/ebeard/Documents/darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    # Handling output layers based on the format of getUnconnectedOutLayers
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if unconnected_out_layers.ndim == 1:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    else:
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]

    cap = cv2.VideoCapture(video_path)
    frame_index = 0
    aoi_data = []

    video_name = os.path.basename(video_path).split('.mp4')[0]

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(os.path.join(output_folder, video_name, "frames")):
        os.makedirs(os.path.join(output_folder, video_name, "frames"))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index % skip_frames == 0:  # Process this frame
            height, width, channels = frame.shape
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        if classes[class_id].lower() == aoi_object: 
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            found_phone = False
            if len(indexes) > 0:  # Check if indexes is not empty
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    aoi_data.append({'frame': frame_index, 'x': x, 'y': y, 'width': w, 'height': h})
                    found_phone = True

            if found_phone:
                cv2.imwrite(os.path.join(output_folder, video_name, "frames", f"frame_{frame_index}.jpg"), frame)

        frame_index += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    if ' ' in aoi_object:
        aoi_object = aoi_object.replace(' ', '')

    with open(os.path.join(output_folder, video_name, f'{aoi_object}_rectangle_data.csv'), 'w', newline='') as csvfile: 
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in aoi_data:
            writer.writerow(data)

# Example usage
#aois = detect_and_save_smartphones(video, 'full_frame', skip_frames=100)
#television = detect_and_save_smartphones(video_path = os.path.join('data', 'sub-019_tv-lego-42b0cb7d', '09e25ad2_0.0-784.762.mp4'), output_folder='full_frame_tv', skip_frames=200)
